In [ ]:
pip install transformers datasets torch sentencepiece

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import AlbertModel, AlbertTokenizer, get_scheduler
from datasets import load_dataset

# Configuration
MODEL_NAME = "albert-base-v2"
BATCH_SIZE = 16
EPOCHS = 3
LR = 2e-5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Tokenizer
tokenizer = AlbertTokenizer.from_pretrained(MODEL_NAME)

# Load TREC Dataset
dataset = load_dataset("trec")
labels_list = dataset["train"].features["coarse_label"].names  # Class names
NUM_CLASSES = len(labels_list)  # 6 classes

# Custom Dataset Class
class TrecDataset(Dataset):
    def __init__(self, split):
        self.data = dataset[split]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data[idx]["text"]
        label = self.data[idx]["coarse_label"]
        inputs = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_tensors="pt")
        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long)
        }

# Dataloaders
train_loader = DataLoader(TrecDataset("train"), batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(TrecDataset("test"), batch_size=BATCH_SIZE)

# Custom ALBERT Model
class CustomAlbertClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.albert = AlbertModel.from_pretrained(MODEL_NAME)
        self.classifier = nn.Linear(self.albert.config.hidden_size, num_classes)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.albert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]  # [CLS] token output
        return self.classifier(pooled_output)

# Initialize Model
model = CustomAlbertClassifier(NUM_CLASSES).to(DEVICE)
optimizer = optim.AdamW(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()
scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_loader) * EPOCHS)

# Training Loop
def train():
    model.train()
    for epoch in range(EPOCHS):
        total_loss, correct, total = 0, 0, 0
        for batch in train_loader:
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels = batch["label"].to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

        scheduler.step()
        print(f"Epoch {epoch+1}: Loss={total_loss/len(train_loader):.4f}, Accuracy={correct/total:.4f}")

# Evaluation Function
def evaluate():
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels = batch["label"].to(DEVICE)
            outputs = model(input_ids, attention_mask)
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)
    print(f"Test Accuracy: {correct / total:.4f}")

# Run Training and Evaluation
train()
evaluate()


C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shaif\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Generating train split:   0%|          | 0/5452 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/500 [00:00<?, ? examples/s]

Epoch 1: Loss=0.4550, Accuracy=0.8465
Epoch 2: Loss=0.4757, Accuracy=0.8208
Epoch 3: Loss=0.2379, Accuracy=0.9382
Test Accuracy: 0.9540


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import AlbertTokenizer, AlbertConfig
from datasets import load_dataset

# Config
MODEL_NAME = "albert-base-v2"
BATCH_SIZE = 16
EPOCHS = 3
LR = 2e-5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenizer
tokenizer = AlbertTokenizer.from_pretrained(MODEL_NAME)

# Load TREC Dataset
dataset = load_dataset("trec")
NUM_CLASSES = len(dataset["train"].features["coarse_label"].names)

# Custom Dataset
class TrecDataset(Dataset):
    def __init__(self, split):
        self.data = dataset[split]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        text = self.data[idx]["text"]
        label = self.data[idx]["coarse_label"]
        inputs = tokenizer(text, padding="max_length", truncation=True, max_length=64, return_tensors="pt")
        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long)
        }

train_loader = DataLoader(TrecDataset("train"), batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(TrecDataset("test"), batch_size=BATCH_SIZE)

# Custom Attention
class DynamicEnergyRegulatedTemporalSpatialAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super().__init__()
        self.d_model = d_model
        self.n_heads = n_heads
        self.head_dim = d_model // n_heads

        self.q_proj = nn.Linear(d_model, d_model)
        self.k_proj = nn.Linear(d_model, d_model)
        self.v_proj = nn.Linear(d_model, d_model)
        self.out_proj = nn.Linear(d_model, d_model)

        self.W_phi = nn.Parameter(torch.randn(d_model, d_model))
        self.W_alpha = nn.Linear(d_model, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, E_embed, mask=None):
        B, L, _ = x.shape

        Q = self.q_proj(x).view(B, L, self.n_heads, self.head_dim).transpose(1, 2)
        K = self.k_proj(x).view(B, L, self.n_heads, self.head_dim).transpose(1, 2)
        V = self.v_proj(x).view(B, L, self.n_heads, self.head_dim).transpose(1, 2)

        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.head_dim ** 0.5)

        phi_E = torch.matmul(E_embed, self.W_phi)
        alpha = self.sigmoid(self.W_alpha(phi_E)).squeeze(-1)
        bias_scalar = phi_E.mean(dim=-1)

        bias = (alpha * bias_scalar).unsqueeze(1).unsqueeze(-1)  # (B, 1, L, 1)
        attn_scores += bias  # broadcast to (B, heads, L, L)

        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask[:, None, None, :] == 0, float('-inf'))

        attn_weights = torch.softmax(attn_scores, dim=-1)
        out = torch.matmul(attn_weights, V)
        out = out.transpose(1, 2).contiguous().view(B, L, self.d_model)
        return self.out_proj(out)

# Encoder Block
class CustomAlbertBlock(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.attn = DynamicEnergyRegulatedTemporalSpatialAttention(config.hidden_size, config.num_attention_heads)
        self.attn_ln = nn.LayerNorm(config.hidden_size)
        self.ffn = nn.Sequential(
            nn.Linear(config.hidden_size, config.intermediate_size),
            nn.GELU(),
            nn.Linear(config.intermediate_size, config.hidden_size)
        )
        self.ffn_ln = nn.LayerNorm(config.hidden_size)

    def forward(self, x, E_embed, mask=None):
        attn_out = self.attn(x, E_embed, mask)
        x = self.attn_ln(x + attn_out)
        ffn_out = self.ffn(x)
        x = self.ffn_ln(x + ffn_out)
        return x

# Custom Encoder

class CustomAlbertEncoder(nn.Module):
    def __init__(self, config, num_layers=12):
        super().__init__()
        self.embeddings = nn.Embedding(config.vocab_size, config.hidden_size)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)

        self.layernorm = nn.LayerNorm(config.hidden_size)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

        self.blocks = nn.ModuleList([CustomAlbertBlock(config) for _ in range(num_layers)])

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, E_embed=None):
        B, L = input_ids.size()
        position_ids = torch.arange(0, L, device=input_ids.device).unsqueeze(0).expand(B, L)
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)

        x = self.embeddings(input_ids) + \
            self.position_embeddings(position_ids) + \
            self.token_type_embeddings(token_type_ids)

        x = self.layernorm(x)
        x = self.dropout(x)

        if E_embed is None:
            E_embed = x

        for block in self.blocks:
            x = block(x, E_embed, attention_mask)
        return x


class CustomAlbertForClassification(nn.Module):
    def __init__(self, config, num_classes):
        super().__init__()
        self.encoder = CustomAlbertEncoder(config, num_layers=12)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.norm = nn.LayerNorm(config.hidden_size)
        self.classifier = nn.Linear(config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, E_embed=None):
        x = self.encoder(input_ids, token_type_ids, attention_mask, E_embed)
        cls_token = x[:, 0]
        cls_token = self.norm(cls_token)
        cls_token = self.dropout(cls_token)
        return self.classifier(cls_token)


# Load config and model
config = AlbertConfig.from_pretrained(MODEL_NAME)
model = CustomAlbertForClassification(config, NUM_CLASSES).to(DEVICE)

# Optimizer & Loss
optimizer = optim.AdamW(model.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()

# Training
def train():
    model.train()
    for ep in range(EPOCHS):
        total, correct, total_loss = 0, 0, 0
        for batch in train_loader:
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels = batch["label"].to(DEVICE)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

        print(f"Epoch {ep+1}: Loss={total_loss/len(train_loader):.4f}, Accuracy={correct/total:.4f}")

# Evaluation
def evaluate():
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(DEVICE)
            attention_mask = batch["attention_mask"].to(DEVICE)
            labels = batch["label"].to(DEVICE)

            outputs = model(input_ids, attention_mask)
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)

    print(f"Test Accuracy: {correct/total:.4f}")

# Run training and evaluation
train()
evaluate()
